Установка библиотеки

In [2]:
!pip install eyetrackfatigue


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 MB 12.4 MB/s eta 0:00:00
  Created wheel for eyetrackfatigue: filename=EyeTrackFatigue-1.0.6-py3-none-any.whl size=43519 sha256=560d57dfb4af363882465b14040598f5e948de3121f79937665fe14fc08cfcae
  Stored in directory: /root/.cache/pip/wheels/27/02/18/5ee5baa5bd50cf8d1e5c3d6c675d9e5992790deb3bcb9c1394
Successfully built eyetrackfatigue


Подключение библиотеки

In [5]:
import EyeTrackFatigue as etf
import EyeTrackFatigue.UI.DataGather as sss


Работа с библиотекой:

In [6]:
section = etf.Input.read_csv_file('test_file.csv') # считывание файла с сырыми данными
metrics = etf.Analise.ParsedData.ParsedData()
metrics.parse(section, 0, 1.0)
metrics.calc_metrics()
metrics.to_csv('test1_output.csv', 'test')

FileNotFoundError: [Errno 2] No such file or directory: 'test_file.csv'

In [7]:
sy = etf.DeviceManager.Synchronizer.Synchronizer()

TypeError: Synchronizer.__init__() missing 1 required positional argument: 'path'